In [42]:
import pandas as pd

# 美國球賽的小費數據集
tips = pd.read_csv('tips.csv')
tips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   total_bill  244 non-null    float64
 1   tip         244 non-null    float64
 2   smoker      244 non-null    object 
 3   day         244 non-null    object 
 4   time        244 non-null    object 
 5   size        244 non-null    int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 11.6+ KB


In [43]:
# 改變tips的欄位名稱
tips.columns = ['總票價', '小費', '是否吸煙', '日期', '時間', '人數']
tips

,總票價,小費,是否吸煙,日期,時間,人數
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4
...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3
240,27.18,2.00,Yes,Sat,Dinner,2
241,22.67,2.00,Yes,Sat,Dinner,2
242,17.82,1.75,No,Sat,Dinner,2


In [44]:
#重新排序欄位的順序
tips = tips[['日期', '時間', '總票價', '小費', '是否吸煙', '人數']]

#新增一個欄位:  小費百分比 = 小費 / 總票價 * 100
tips['小費百分比'] = tips['小費'] / tips['總票價'] * 100

tips

C:\Users\alunt\AppData\Local\Temp\ipykernel_18384\3678449229.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tips['小費百分比'] = tips['小費'] / tips['總票價'] * 100


,日期,時間,總票價,小費,是否吸煙,人數,小費百分比
0,Sun,Dinner,16.99,1.01,No,2,5.944673
1,Sun,Dinner,10.34,1.66,No,3,16.054159
2,Sun,Dinner,21.01,3.50,No,3,16.658734
3,Sun,Dinner,23.68,3.31,No,2,13.978041
4,Sun,Dinner,24.59,3.61,No,4,14.680765
...,...,...,...,...,...,...,...
239,Sat,Dinner,29.03,5.92,No,3,20.392697
240,Sat,Dinner,27.18,2.00,Yes,2,7.358352
241,Sat,Dinner,22.67,2.00,Yes,2,8.822232
242,Sat,Dinner,17.82,1.75,No,2,9.820426


In [45]:
#依據日期建立一個群組
grouped = tips.groupby('日期')
grouped[['小費']].mean()


,小費
日期,
Fri,2.734737
Sat,2.993103
Sun,3.255132
Thur,2.771452


In [46]:
#依據日期和是否吸菸,建立一個群組
grouped1 = tips.groupby(['日期', '是否吸煙'])

#取得小費的平均值
s1 = grouped1['小費'].mean()

#s1是一個多層索引的Series
#變成DataFrame
df1 = s1.unstack()
df1

#s1我要unstack，日期在欄位，是否吸菸在索引
df2 = s1.unstack(level= '日期' )
df2


日期,Fri,Sat,Sun,Thur
是否吸煙,,,,
No,2.8125,3.102889,3.167895,2.673778
Yes,2.7140,2.875476,3.516842,3.030000


In [47]:
grouped2 = tips.groupby(['日期', '是否吸煙'])
df3 = grouped2['小費'].agg(['mean','sum','count']).unstack(level='日期')
df3.columns.names = ['統計量','日期']

#欄位統計量和日期欄的交換位置
df3.swaplevel(axis=1)


日期,Fri,Sat,Sun,Thur,Fri,Sat,Sun,Thur,Fri,Sat,Sun,Thur
統計量,mean,mean,mean,mean,sum,sum,sum,sum,count,count,count,count
是否吸煙,,,,,,,,,,,,
No,2.8125,3.102889,3.167895,2.673778,11.25,139.63,180.57,120.32,4,45,57,45
Yes,2.7140,2.875476,3.516842,3.030000,40.71,120.77,66.82,51.51,15,42,19,17


In [ ]:
grouped2 = tips.groupby(['日期', '是否吸煙'])
# mean, sum, count,我要改為中文名稱平均, 總和, 筆數
df5 = grouped2[['小費', '總票價']].agg([('平均', 'mean'), ('總和', 'sum'), ('筆數', 'count')])
df5

In [61]:

# 自訂聚合函數,最大值-最小值
def my_range(group):
    return group.max() - group.min()

grouped3 = tips.groupby(['日期', '是否吸煙'])
print(len(grouped3))
df6 = grouped3[['小費']].agg([('平均', 'mean'),('最大值和最小值差', my_range)])

8


In [62]:

# 取得df6的小費欄位和平均欄位
df6['小費']['平均']

日期    是否吸煙
Fri   No      2.812500
      Yes     2.714000
Sat   No      3.102889
      Yes     2.875476
Sun   No      3.167895
      Yes     3.516842
Thur  No      2.673778
      Yes     3.030000
Name: 平均, dtype: float64

In [63]:
# df6取得日期索引
df6.loc['Sun']

小費         
            平均 最大值和最小值差
是否吸煙                   
No    3.167895     4.99
Yes   3.516842     5.00